## Лабораторная работа №2 (курс "Математические методы анализа текстов")


#### Тема: Языковое моделирование и определение языка.


**Выдана**:   13 марта 2017

**Дедлайн**:   <font color='red'>9:00 утра 26 марта 2017</font>

**Среда выполнения**: Jupyter Notebook (Python 2.7)

#### Правила:

Результат выполнения задания $-$ отчет в формате Jupyter Notebook с кодом и выводами. В ходе выполнения задания требуется реализовать все необходимые алгоритмы, провести эксперименты и ответить на поставленные вопросы. Дополнительные выводы приветствуются. Чем меньше кода и больше комментариев $-$ тем лучше.

Все ячейки должны быть "выполненными", при этом результат должен воспроизвдиться при проверке (на Python 2.7). Если какой-то код не был запущен или отрабатывает с ошибками, то пункт не засчитывается. Задание, сданное после дедлайна, _не принимается_. Совсем.


Задание выполняется самостоятельно. Вы можете обсуждать идеи, объяснять друг другу материал, но не можете обмениваться частями своего кода. Если какие-то студенты будут уличены в списывании, все они автоматически получат за эту работу 0 баллов, а также предвзято негативное отношение семинаристов в будущем. Если вы нашли в Интернете какой-то код, который собираетесь заимствовать, обязательно укажите это в задании: вполне вероятно, что вы не единственный, кто найдёт и использует эту информацию.

#### Постановка задачи:

В данной лабораторной работе Вам предстоит реализовать n-грамную языковую модель с несколькими видами сглаживания:
- Add-one smoothing
- Stupid backoff
- Interpolation smoothing
- Kneser-Ney smoothing

Вы обучите ее на готовых корпусах, оцените качество и проведете ряд экспериментов. Во второй части задания Вы примените реализованную модель (но с буквенными n-граммами) к задаче распознавания языка. 


# Языковые модели

Цель языкового моделирования заключается в том, чтобы присвоить некоторые вероятности предложениям. Задача состоит в подсчете вероятности $P(W) = P(w_1, \dots, w_n)$ или $P(w_n \mid w_1, \dots, w_{n-1})$. Модель, умеющая вычислять хотя бы одну из этих двух вероятностей, называется **языковой моделью** (LM от Language Model).

Согласно **цепному правилу** (chain rule):

$$P(X_1, \dots, X_n) = P(X_1)P(X_2 \mid X_1)\dots P(X_n \mid X_1, \dots, X_{n-1}).$$ 

Также мы знаем, что

$$
    P(X_n \mid X_1, \dots, X_{n-1}) = \frac{P(X_1, \dots, X_n)}{P(X_1, \dots, X_{n-1})},
$$

следовательно, для того чтобы оценить $P(X_n \mid X_1, \dots, X_{n-1})$ нужно посчитать $P(X_1, \dots, X_n)$ и $P(X_1, \dots, X_{n-1})$. Но эти вероятности будут чрезвычайно малы, если мы возьмем большое $n$, так множество предложений из $n$ слов растет экспоненциально. Для упрощения применим **марковское предположение**: 

$$P(X_n \mid X_1, \dots, X_{n-1}) = P(X_n \mid X_{n - k + 1}, \dots, X_{n-1})$$

для некоторого фиксированного (небольшого) $k$. Это предположение говорит о том, что $X_{n}$ не зависит от $X_{1}, \dots, X_{n - k}$, то есть на следующее слово влияет лишь контекст из предыдущих $k - 1$ слова. Таким образом, мы получаем финальную вероятность:

$$
    P(w_1, \dots, w_n) = \prod_i P(w_i \mid w_{i-k+1}, \dots, w_{i - 1}).
$$

Далее для краткости будем обозначать $w_{i-k}^i := w_{i-k}, \dots, w_{i}$.

## Хранилище n-грам

Для начала выполним вспомогательную работу. Следуйте комментариям, чтобы написать NGramStorage с удобным интерфейсом.

In [1]:
import re
import math
import random
from collections import Counter
import numpy as np
from sklearn.cross_validation import train_test_split

C:\Users\ASUS\Anaconda3\envs\py27\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
class NGramStorage:
    """Storage for ngrams' frequencies.
    
    Args:
        sents (list[list[str]]): List of sentences from which ngram
            frequencies are extracted.
        max_n (int): Upper bound of the length of ngrams.
            For instance if max_n = 2, then storage will store
            0, 1, 2-grams.
            
    Attributes:
        max_n (Readonly(int)): Upper bound of the length of ngrams.
    """
        
    def __init__(self, sents=[], max_n=0):
        self.__max_n = max_n
        self.__ngrams = {i: Counter() for i in xrange(self.__max_n + 1)}
        # self._ngrams[K] should have the following interface:
        # self._ngrams[K][(w_1, ..., w_K)] = number of times w_1, ..., w_K occured in words
        # self._ngrams[0][()] = number of all words
        ### YOUR CODE HERE
        for sent in sents:
            slices = [sent[i:] for i,_ in enumerate(sent)]
            #print(slices)
            self.__ngrams[0].update({tuple(): len(sent)})
            for i in xrange(1, self.__max_n + 1):
                if i > len(sent):
                    break
                self.__ngrams[i].update(zip(*slices[:i]))
        ### END YOUR CODE
        
    def add_unk_token(self):
        """Add UNK token to 1-grams."""
        # In order to avoid zero probabilites 
        # здесь был баг из-за которого токены добавлялись бесконечно
        if self.__max_n == 0 or self.__ngrams[1].get((u'UNK', )) is not None:
            return
        self.__ngrams[0][()] += 1
        self.__ngrams[1][(u'UNK',)] = 1
        
    @property
    def max_n(self):
        """Get max_n"""
        return self.__max_n
        
    def __getitem__(self, k):
        """Get dictionary of k-gram frequencies.
        
        Args:
            k (int): length of returning ngrams' frequencies.
            
        Returns:
            Dictionary (in fact Counter) of k-gram frequencies.
        """
        # Cheking the input
        if not isinstance(k, int):
            raise TypeError('k (length of ngrams) must be an integer!')
        if k > self.__max_n:
            raise ValueError('k (length of ngrams) must be less or equal to the maximal length!')
        return self.__ngrams[k]
    
    def __call__(self, ngram):
        """Return frequency of a given ngram.
        
        Args:
            ngram (tuple): ngram for which frequency should be computed.
            
        Returns:
            Frequency (int) of a given ngram.
        """
        # Cheking the input
        if not isinstance(ngram, tuple):
            raise TypeError('ngram must be a tuple!')
        if len(ngram) > self.__max_n:
            raise ValueError('length of ngram must be less or equal to the maximal length!')
        if len(ngram) == 1 and ngram not in self.__ngrams[1]:
            return self.__ngrams[1][(u'UNK', )]
        return self.__ngrams[len(ngram)][ngram]

Посмотрим, как это все работает на модельных примерах.

In [3]:
ngs = NGramStorage([['he', 'is', 'cat', 'he', 'is'], ['little', 'boy', 'is', 'little']], 6)

In [4]:
ngs[2]

Counter({('boy', 'is'): 1,
         ('cat', 'he'): 1,
         ('he', 'is'): 2,
         ('is', 'cat'): 1,
         ('is', 'little'): 1,
         ('little', 'boy'): 1})

In [5]:
ngs.add_unk_token()

In [6]:
ngs[1], ngs[0]

(Counter({(u'UNK',): 1,
          ('boy',): 1,
          ('cat',): 1,
          ('he',): 2,
          ('is',): 3,
          ('little',): 2}),
 Counter({(): 10}))

## Оценка качества

Скачайте brown корпус, обучите модель и протестируйте на нескольких примерах последовательностей.

In [7]:
import nltk
# Uncomment next row and download brown corpus
# nltk.download()
from nltk.corpus import brown

In [8]:
all_sents = list(brown.sents())
random.shuffle(all_sents)
print('Number of all sentences = {}'.format(len(all_sents)))
train_sents = all_sents[:int(0.8 * len(all_sents))]
test_sents = all_sents[int(0.8 * len(all_sents)):]
print('Number of train sentences = {}'.format(len(train_sents)))
print('Number of test sentences = {}'.format(len(test_sents)))

Number of all sentences = 57340
Number of train sentences = 45872
Number of test sentences = 11468


In [9]:
# Create storage of 0, 1, 2, 3-grams
storage = NGramStorage(train_sents, 3)

In [10]:
# It's time to test your code
print(storage(('to', 'be')))
print(storage(('or',)))
print(storage(('not', 'to', 'be')))
print(storage(('somethingweird',)))
print(storage(()))

1351
3278
29
0
928571


In [11]:
print(train_sents[0])

[u'The', u'dog', u'refused', u'to', u'be', u'scared', u'off', u',', u'so', u'Kahler', u'had', u'purchased', u'some', u'small', u'firecrackers', u'.']


Для численного измерения качества языковой модели определим **перплексию**:

$$
    {\mathbb{P}}(w_1, \dots, w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_i P(w_i \mid w_{i - k}, \dots, w_{i - 1})\right)^{-\frac1N},
$$

Вижно, что минимизация перплексии эквивалентна максимизации правдоподобия модели.

Реализуйте функцию по подсчету перплексии. Обратите внимание, что перплексия по корпусу равна произведению вероятностей **всех** предложений в степени $-\frac1N$, где $N -$ суммарная длина всех предложений.

In [12]:
def perplexity(estimator, sents):
    '''Estimate perplexity of the sequence of words using prob_estimator.'''
    ### YOUR CODE HERE
    # Avoid log(0) by replacing zero by 10 ** (-50).
    #perp = 100
    N = 0
    log_prob = 0.
    log_small_val = math.log(10.**(-50))
    for sent in sents:
        #str_sent = [str(word) for word in sent]
        str_sent = sent
        p = estimator.prob(str_sent)
        log_prob += math.log(p) if p > 10.**(-50) else log_small_val
        N += len(str_sent)
    log_prob *= -1.0/N
    perp = math.exp(log_prob)
    #perp = log_prob
    ### END YOUR CODE
    
    return perp

## Оценка вероятностей n-грам

Первый и простейший способ оценки вероятностей N-грам следующий:

$$
    \hat P_{S}(w_{N} \mid w_1^{N - 1}) = \frac{c(w_1^N)}{c(w_1^{N-1})}.
$$

где $c(w_1^N)$ — это число последовательностей $w_1, \dots, w_N$ в корпусе, $S$ символизирует Straightforward. 

In [13]:
class StraightforwardProbabilityEstimator:
    """Class for simplest probability estimations of type P(word | context).
    
    P(word | context) = c(context + word) / c(context), where
    c(sequence) - number of occurances of the sequence in the corpus.
    
    Args:
        storage(NGramStorage): Object of NGramStorage class which will
            be used to extract frequencies of ngrams.
    """
    
    def __init__(self, storage):
        self.__storage = storage
        # Adding UNK token to avoid zero probabilities
        self.__storage.add_unk_token()
        
    def cut_context(self, context):
        """Cut context if it is too large.
        
        Args:
            context (tuple[str]): Some sequence of words.
        
        Returns:
            Cutted context (tuple[str]) up to the length of max_n.
        """
        if self.__storage.max_n == 1:
            return ()
        if len(context) + 1 > self.__storage.max_n:
            context = context[-self.__storage.max_n + 1:]
        return context
        
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        # Cheking the input
        if not isinstance(word, basestring):
            raise TypeError('word must be a string!')
        if not isinstance(context, tuple):
            raise TypeError('word must be a string!')
        # If context is too large, let's cut it.
        context = self.cut_context(context)
        phrase_counts = self.__storage(context + (word, ))
        context_counts = self.__storage(context)
        # Avoiding 0 / 0.
        if context_counts == 0:
            return 0.
        return 1. * phrase_counts / context_counts
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.
        
        Args:
            sentence (list[str]): Sentence for probability estimation.
            
        Returns:
            Probability (float) P(sentence).
        """
        prob = 1.
        for i in range(len(sent)):
            prob *= self(sent[i], tuple(sent[:i]))
        return prob

Во всех классах в проверке if not isinstance(word, str) хорошо заменить str на basestring, чтобы не было проблем со строками юникода.

In [14]:
# Initialize estimator
simple_estimator = StraightforwardProbabilityEstimator(storage)

# Estimating perplexity
print('Simple estimator perplexity = {}'.format(perplexity(simple_estimator, test_sents)))
print(simple_estimator.prob('To be'.split()))
print(simple_estimator.prob('To be or not to be'.split()))

Simple estimator perplexity = 252.860056335
1.50769137988e-05
0.0


Посчитаем перплексию униграмной модели.

In [15]:
uni_storage = NGramStorage(train_sents, 1)
uni_simple_estimator = StraightforwardProbabilityEstimator(uni_storage)
print('Simple estimator perplexity = {}'.format(perplexity(uni_simple_estimator, test_sents)))
print(uni_simple_estimator.prob('To be'.split()))
print(uni_simple_estimator.prob('To be or not to be'.split()))

Simple estimator perplexity = 104.592289179
2.01298710828e-06
3.31621181134e-15


Ответьте на следующие вопросы (внутри ipython ноутбука):

**Q:** Какие выводы можно сделать? Почему $P(\text{To be or not to be}) = 0$, хотя мы и добавили UNK токен?  
**A:** Мы добавили такой токен только для униграм, следовательно, вероятности биграм и триграм все так же могут быть равными нулю, что в произведении даст ноль. Видно, что в униграмной модели эта вероятность выше.

**Q:** Почему перплексия униграмной модели меньше, чем триграмной?  
**A:** Вероятность встретить униграму выше, чем триграму (ну по крайней мере обычно). Следовательно и произведение вероятностей будет выше, а перплексия, соответственно, ниже.


## Add-one smoothing

Простейший вид сглаживания — **сглаживание Лапласа**. Чтобы избавиться от нулевых вероятностей $P(w_{N} \mid w_1^{N - 1})$, будем использовать формулу:

$$
    \hat P_{AOS}(w_{N} \mid w_1^{N - 1}) = \frac{c(w_1^N) + \delta}{c(w_1^{N-1}) + \delta V},
$$

где $V$ — это размер словаря, а $\delta$ — некоторая фиксированная константа.

Реализуйте класс, осуществляющий сглаживание Лапласа. Он должен иметь аналогичный интерфейс, как и StraightforwardProbabilityEstimator.

In [16]:
class LaplaceProbabilityEstimator:
    """Class for probability estimations of type P(word | context).
    
    P(word | context) = (c(context + word) + delta) / (c(context) + delta * V), where
    c(sequence) - number of occurances of the sequence in the corpus,
    delta - some constant,
    V - number of different words in corpus.
    
    Args:
        storage(NGramStorage): Object of NGramStorage class which will
            be used to extract frequencies of ngrams.
        delta(float): Smoothing parameter.
    """
    
    def __init__(self, storage, delta=1.):
        self.__storage = storage
        self.__delta = delta
        #если V - константа для корпуса, то почему бы ей не быть тут?
        self.__V = len(self.__storage[1])
        
    def cut_context(self, context):
        """Cut context if it is too large.
        
        Args:
            context (tuple[str]): Some sequence of words.
        
        Returns:
            Cutted context (tuple[str]) up to the length of max_n.
        """
        if len(context) + 1 > self.__storage.max_n:
            context = context[-self.__storage.max_n + 1:]
        return context
        
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        # Cheking the input
        if not isinstance(word, basestring):
            raise TypeError('word must be a string!')
        if not isinstance(context, tuple):
            raise TypeError('context must be a tuple!')
            
        ### YOUR CODE HERE
        prob = 1.
        # If context is too large, let's cut it.
        context = self.cut_context(context)
        phrase_counts = self.__storage(context + (word, ))
        context_counts = self.__storage(context)
        # Avoiding 0 / 0.
        denominator = float(context_counts + self.__delta*self.__V)
        # 1e-20 вместо 0, потому что машинная точность
        if denominator < 1e-20:
            return 0.
        return (phrase_counts + self.__delta) / denominator
        ### END YOUR CODE
            
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.
        
        Args:
            sentence (list[str]): Sentence for probability estimation.
            
        Returns:
            Probability (float) P(sentence).
        """
        prob = 1.
        for i in range(len(sent)):
            prob *= self(sent[i], tuple(sent[:i]))
        return prob

Подберите наилучший параметр $\delta$ для данного корпуса.

Подберем лучший параметр на отложенной выборке каким-нибудь методом оптимизации.

In [17]:
%%time
# Try to find out best delta parametr. We will not provide you any strater code.
### YOUR CODE HERE
from scipy.optimize import minimize_scalar
f = lambda best_delta: perplexity(LaplaceProbabilityEstimator(storage, best_delta), test_sents)
#придется подождать ¯\_(ツ)_/¯
res = minimize_scalar(f, method='brent')
best_delta = res.x
print('Best delta = {}'.format(best_delta))
### END YOUR CODE

# Initialize estimator
laplace_estimator = LaplaceProbabilityEstimator(storage, best_delta)

# Let's make some estimations
print('Laplace estimator perplexity = {}'.format(perplexity(laplace_estimator, test_sents)))
print(laplace_estimator.prob('To be'.split()))
print(laplace_estimator.prob('To be or not to be'.split()))

Best delta = 0.000385776530282
Laplace estimator perplexity = 131.907626439
1.42618166483e-05
1.59868803672e-17
Wall time: 46.2 s


## Stupid backoff

Идея **простого отката** довольно понятна. Если у нас есть достаточно информцаии для подсчета вероятности $k$-грам, то будем использовать $k$-грамы. Иначе будем использовать вероятности $(k-1)$-грам с некоторым множителем, например, $0.4$, и так далее. К сожалению, в данном случае мы получим не вероятностное распределение, но в большинстве задач это не имеет принципиального значения. Если это все же важно, то необходимо подобрать множитель соответствующим образом.

Реализуйте класс, симулирующий сглаживание простым откатом. Он должен иметь аналогичный интерфейс, как и StraightforwardProbabilityEstimator.

In [18]:
class StupidBackoffProbabilityEstimator:
    """Class for stupid backoff probability estimations.
    
    P(word | context) =
        P'(word | context),                  if  P'(word | context) > 0;
        P'(word | context[1:]) * multiplier, if  P'(word | context) == 0
                                             and P'(word | context[1:]) > 0;
        ...
    P'(word | context) - probability of a word provided context of a base estimator.
    
    Args:
        base_estimator(BaseProbabilityEstimator): Object of BaseProbabilityEstimator
            or some other class which can estimate conditional probabilities.
        multiplier (float): Multiplier which is used for probability estimations.
    """
    
    def __init__(self, base_estimator, multiplier=0.1):
        self.__base_estimator = base_estimator
        self.__mult = multiplier
        
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        
        ### YOUR CODE HERE
        
        prob = self.__base_estimator(word, context)
        if prob < 1e-20 and len(context) != 0:
            return self.__mult * self.__call__(word, context[1:])
        else:
            return prob
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.
        
        Args:
            sentence (list[str]): Sentence for probability estimation.
            
        Returns:
            Probability (float) P(sentence).
        """
        prob = 1.
        for i in range(len(sent)):
            prob *= self(sent[i], tuple(sent[:i]))
        return prob

In [19]:
# Initialize estimator
sbackoff_estimator = StupidBackoffProbabilityEstimator(simple_estimator, .4)

# Let's make some estimations
print('Stupid backoff estimator perplexity = {}'.format(perplexity(sbackoff_estimator, test_sents)))
print(sbackoff_estimator.prob('To be'.split()))

Stupid backoff estimator perplexity = 119.146522426
1.50769137988e-05


Ответьте на следующие вопросы (внутри ipython ноутбука):

**Q:** Почему бессмысленно измерять перплексию в случае **Stupid backoff**?  
**A:** Потому что этот метод не дает корректного распределения вероятностей. Соответственно, мы перемножаем не вероятности предложений, а просто какие неотнормированные числа, характеризующие предложения. Поэтому сравнивать перплексию Stupid backoff и других методов бессмысленно. Однако между разными Stupid backoff минимизация перплексии уже имеет смысл.


## Interpolation smoothing

В данном случае идея сглаживания посредством **интерполяции** также крайне проста. Пусть у нас есть $N$-грамная модель. Заведем вектор $\bar\lambda = (\lambda_1, \dots, \lambda_N)$, такой, что $\sum_i\lambda_i = 1$ и $\lambda_i \geq 0$. Тогда

$$
    \hat P_{IS}(w_{N} \mid w_1^{N-1}) = \sum_{i=1}^N \lambda_i \hat P_{S}(w_N \mid w_{N-i+1}^{N-1}).
$$

Придумайте, как обойтись одним вектором $\bar\lambda$, т.е. пользоваться им как в случае контекста длины $N$, так и при контексте меньшей длины (например, в начале предложения). Если мы просто обрубим сумму, то у нас уже не будет вероятностное распределение, что, конечно же, плохо.

In [20]:
class InterpolationProbabilityEstimator:
    """Class for interpolation probability estimations.
    
    P(word | context) =
        lambda_N * P'(word | context) +
        lambda_{N-1} * P'(word | context[1:]) +
        ... +
        lambda_1 * P'(word)
    P'(word | context) - probability of a word provided context of a base estimator.
    
    Args:
        base_estimator(BaseProbabilityEstimator): Object of BaseProbabilityEstimator
            or some other class which can estimate conditional probabilities.
        lambdas (np.array[float]): Lambdas which are used for probability estimations.
    """
    
    def __init__(self, base_estimator, lambdas):
        self.lambdas = lambdas
        self.__base_estimator = base_estimator
        
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        
        ### YOUR CODE HERE
        prob = 0.
        trunc_context = context[-len(self.lambdas)+1:]
        for i,_ in enumerate(self.lambdas):
            #print(self.lambdas[-(i+1)], word, trunc_context[i:])
            prob += self.lambdas[-(i+1)]*self.__base_estimator(word, trunc_context[i:])
        ### END YOUR CODE
        return prob
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.
        
        Args:
            sentence (list[str]): Sentence for probability estimation.
            
        Returns:
            Probability (float) P(sentence).
        """
        prob = 1.
        for i in range(len(sent)):
            prob *= self(sent[i], tuple(sent[:i]))
        return prob

Если длина контекста слишком мала, то всем $\lambda_i$, которым не хватило n-грам, будут использовать униграмную модель. В итоге все равно получим корректное распределение вероятностей. Это простейшее решение, возможно перевзвешивание работает лучше, но возникает вопрос - как удачнее распределить веса.

In [21]:
import numpy as np

# Initialize estimator
interpol_estimator = InterpolationProbabilityEstimator(simple_estimator, np.array([0.2, 0.2, 0.6]))

# Let's make some estimations
print('Interpolation estimator perplexity = {}'.format(perplexity(interpol_estimator, test_sents)))
print(interpol_estimator.prob('To be'.split()))

Interpolation estimator perplexity = 89.1603991062
9.8513431226e-06


Обучить значения параметров $\lambda$ можно с помощью EM-алгоритма, но мы не будем этого здесь делать.

Ради интереса попробуем выполнить оптимизацию на отложенной выборке как для сглаживания Лапласа.

In [22]:
%%time
### YOUR CODE HERE
from scipy.optimize import minimize
f = lambda x: perplexity(InterpolationProbabilityEstimator(simple_estimator, x), test_sents)
cons = ({'type': 'eq', 'fun': lambda x:  np.sum(x) - 1.})
#придется подождать ¯\_(ツ)_/¯
res = minimize(f, np.array([0.2, 0.2, 0.6]), method='SLSQP', constraints=cons, 
               options={'ftol': 1e-6, 'disp': True})
best_x = res.x
print('Best lambdas = {}'.format(best_x))
print('Best perplexity = {}'.format(res.fun))
### END YOUR CODE

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 84.006395659
            Iterations: 7
            Function evaluations: 41
            Gradient evaluations: 7
Best lambdas = [ 0.36881124  0.45051599  0.18067277]
Best perplexity = 84.006395659
Wall time: 2min 33s


## Kneser-Ney smoothing

Идея данного сглаживания заключается в том, что словам, которые участвуют в большом количестве контекстов, присваиваются большие вероятности, а те, которые используются в паре-тройке контекстов, получают маленькие вероятности. Авторы данного сглаживания формализовали это следующим образом. Введем обозначения

$$
    N_{c}(w) := \left|\{\hat w : c(\hat w, w) > 0\}\right|
$$

$-$ число N-грамм, в которых последней частью идёт $w$ (слово или последовательность слов).

Опеределим рекурентное соотношение:

$$
    \hat P_{KN} (w_1) = \frac{N_{c}(w_1)}{\sum_{w} N_{c}(w)},
$$

$$
    \hat P_{KN}(w_{i} \mid w_{i - n + 1}^{i-1}) = \frac{{\rm max}\{c(w_{i -n +1}^i) - \delta, 0\}}{\sum_{w}c(w_{i - n + 1}^{i-1}, w)} + \lambda(w^{i-1}_{i-n+1}) \hat P_{KN}(w_{i} \mid w^{i-1}_{i-n+2}).
$$

где

$$
\lambda(w^{i-1}_{i-n+1}) = \frac{\delta}{\sum_{w}c(w_{i - n + 1}^{i-1}, w)}N_{c}(w_{i-n+1}^{i-1})
$$

$-$ весовой множитель.


Реализуйте данный подход.

In [25]:
class KneserNeyProbabilityEstimator:
    """Class for probability estimations of type P(word | context).
    
    P(word | context) = ...
    
    Args:
        storage(NGramStorage): Object of NGramStorage class which will
            be used to extract frequencies of ngrams.
        delta(float): KneserNey parameter.
    """
    
    def __init__(self, storage, delta=1.):
        self.__storage = storage
        self.__delta = delta
        #значения N_c можно проинициализировать заранее
        self.__nc_dict = dict()
        for i in xrange(self.__storage.max_n):
            for key in self.__storage[storage.max_n]:
                if self.__nc_dict.get(key[-i-1:]):
                    self.__nc_dict[key[-i-1:]] += 1 
                else:
                    self.__nc_dict[key[-i-1:]] = 1
        
    def cut_context(self, context):
        """Cut context if it is too large.
        
        Args:
            context (tuple[str]): Some sequence of words.
        
        Returns:
            Cutted context (tuple[str]) up to the length of max_n.
        """
        if len(context) + 1 > self.__storage.max_n:
            context = context[-self.__storage.max_n + 1:]
        return context
    
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        # Cheking the input
        if not isinstance(word, basestring):
            raise TypeError('word must be a string!')
        if not isinstance(context, tuple):
            raise TypeError('word must be a string!')
        # If context is too large, let's cut it.
        context = self.cut_context(context)
        #print(context, word)
        #print(word, context)

        ### YOUR CODE HERE
        prob = 1.        
        if len(context):
            #denominator - число n-грамм, оканчивающихся на context. Но их будет примерно self.__storage(context)
            #это эвристика, которая экономит кучу времени 
            denominator = self.__storage(context)
            if denominator > 1e-20:
                nc = self.__nc_dict.get(context)
                if nc is None:
                    nc = 0.
                lmb = 1.*self.__delta*nc / denominator
                prob = self(word, context[1:])*lmb + 1.*max(self.__storage(context+(word,))-self.__delta, 0)/denominator
            else:
                prob = self(word, context[1:])
            return prob
        else:
            nc = self.__nc_dict.get((word, ))
            if nc is None:
                return 0.
            prob = 1.*nc / len(self.__storage[self.__storage.max_n])
            return prob
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.
        
        Args:
            sentence (list[str]): Sentence for probability estimation.
            
        Returns:
            Probability (float) P(sentence).
        """
        prob = 1.
        for i in range(len(sent)):
            prob *= self(sent[i], tuple(sent[:i]))
        return prob

In [26]:
%%time
# Initialize estimator
kn_estimator = KneserNeyProbabilityEstimator(storage)

# Estimating perplexity
print('Simple estimator perplexity = {}'.format(perplexity(kn_estimator, test_sents)))
print(kn_estimator.prob('To be'.split()))
print(kn_estimator.prob('To be or not to be'.split()))

Simple estimator perplexity = 135.626142276
1.69276159763e-06
1.21911716984e-13
Wall time: 5.23 s


Видимо, подбор $\delta$ может улучшить результат.

## Определение языка документа

**Постановка задачи:**  
Одна из задач, которая может быть решена при помощи языковых моделей $-$ **определение языка документа**. Реализуйте два классификатора для определения языка документа:
1. Наивный классификатор, который будет учитывать частоты символов и выбирать язык текста по признаку: распределение частот символов "наиболее похоже" на распределение частот символов в выбранном языке.
2. Классификатор на основе языковых моделей. Сами придумайте, как он должен работать.  
_Подсказка_: лучше считать n-грамы не по словам, а по символам.

---

**Как представлены данные:**  
Во всех текстовых файлах на каждой строчке записано отдельное предложение.
1. В папке _data_ находятся две папки: _full_ и _plain_. В _full_ находятся тексты в той форме, что они были взяты из сети, в _plain_ находятся те же самые тексты, но с них сначала была снята диакритика, а затем русский и греческий тексты были транслитерованы в английский.
2. В каждой из папок _full_ и _plain_ находятся папки _train_ и _test_.
3. В _train_ находятся файлы с текстами с говорящими именами, например, _ru.txt_, _en.txt_.
4. В _test_ находятся файлы _1.txt_, _2.txt_, $\dots$ в которых хранятся тексты, язык которых нужно определить. В этой же папке находится файл _ans.csv_, в котором вы можете найти правильные ответы и проверить, насколько хорошо сработали Ваши алгоритмы.

---

**Что нужно сделать:**  
Напишите два своих классификатора (которые описаны в постановке задачи) и получите максимально возможное accuracy на test-сете. Разрешается использовать только _train_ для обучения.

---

**В данном задании мы не предоставляем стартового кода!**

### Naive

#### plain

Подсчитаем сколько раз встречается каждый символ в каждом из языков, а так же скачаем ответы.

In [27]:
# Write the code and estimate accuracy of your method.
# Create your own classifiers.

### YOUR CODE HERE
from zipfile import ZipFile
#tqdm - хороший пакет, использовать его конечно необязательно (но приятно!)
import tqdm
import csv
from nltk.tokenize import word_tokenize
import sys

langs = ['ca', 'de', 'el', 'en', 'eo', 'es', 'fi', 
         'fr', 'hu', 'it', 'nl', 'no', 'pl', 'pt', 'ru', 'sv']  

lang_model_naive = dict()
with ZipFile('language_detection.zip', 'r') as myzip:
    for lang in tqdm.tqdm(langs):
        raw = myzip.read('plain/train/{}.txt'.format(lang))
        lang_model_naive[lang] = Counter(raw)
        lang_model_naive[lang]['\n'] = 0

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:16<00:00,  1.03s/it]


In [28]:
ans = dict()
with ZipFile('language_detection.zip', 'r') as myzip:
    ans_doc = myzip.open('plain/test/ans.csv')
    reader = csv.DictReader(ans_doc, fieldnames=['num', 'lang'])
    for row in reader:
        key = int(row['num'])
        ans[key] = row['lang']
    ans_doc.close()

В качестве метрики будем использовать https://en.wikipedia.org/wiki/Hellinger_distance. Я убрал нормировочные константы и корень из суммы, так как нас интересует данная величина лишь в сравнении, а не само абсолютное значение. Так зачем тогда тратить процессорное время?

In [29]:
def hellinger(counter, lang_model):
    un = set.union(set(counter.keys()), set(lang_model.keys()))
    s = 0
    counter_den = sum(counter.values())
    model_den = sum(lang_model.values())
    for u in un:
        t = math.sqrt(1.*counter[u]/counter_den) - math.sqrt(1.*lang_model[u]/model_den)
        t = t**2
        s += t
    return s

In [30]:
pred = dict()
with ZipFile('language_detection.zip', 'r') as myzip:
    for i in tqdm.tqdm(xrange(1, 240+1)):
        raw = myzip.read('plain/test/{}.txt'.format(i))
        counter = Counter(raw)
        counter['\n'] = 0
        min_dist = sys.float_info.max
        forecast = None
        for lang in langs:
            dist = hellinger(counter, lang_model_naive[lang])
            if min_dist > dist:
                min_dist = dist
                forecast = lang
        pred[i] = forecast

100%|███████████████████████████████████████████████████████████████████████████████| 240/240 [00:01<00:00, 170.45it/s]


In [31]:
errors = 0
for key in ans.keys():
    errors += 1 if ans[key]!=pred[key] else 0
print('Absolute number of errors {} (out of {} possible)'.format(errors, len(ans.keys())))
print('Relative number of errors {}%'.format(1.*errors/len(ans.keys())))

Absolute number of errors 0 (out of 240 possible)
Relative number of errors 0.0%


#### full

Делаем все то же самое. Добавил только перекодировку текстов по сравнению с предыдущим случаем.

In [32]:
lang_model_naive = dict()
with ZipFile('language_detection.zip', 'r') as myzip:
    for lang in tqdm.tqdm(langs):
        raw = myzip.read('full/train/{}.txt'.format(lang))
        lang_model_naive[lang] = Counter(raw.decode('utf-8'))
        lang_model_naive[lang]['\n'] = 0

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:17<00:00,  1.09s/it]


In [33]:
pred = dict()
with ZipFile('language_detection.zip', 'r') as myzip:
    for i in tqdm.tqdm(xrange(1, 240+1)):
        raw = myzip.read('full/test/{}.txt'.format(i))
        counter = Counter(raw.decode('utf-8'))
        counter['\n'] = 0
        min_dist = sys.float_info.max
        forecast = None
        for lang in langs:
            dist = hellinger(counter, lang_model_naive[lang])
            if min_dist > dist:
                min_dist = dist
                forecast = lang
        pred[i] = forecast

100%|███████████████████████████████████████████████████████████████████████████████| 240/240 [00:01<00:00, 150.56it/s]


In [34]:
errors = 0
for key in ans.keys():
    errors += 1 if ans[key]!=pred[key] else 0
print('Absolute number of errors {} (out of {} possible)'.format(errors, len(ans.keys())))
print('Relative number of errors {}%'.format(1.*errors/len(ans.keys())))

Absolute number of errors 0 (out of 240 possible)
Relative number of errors 0.0%


В итоге получаем достаточно неплохой результат в обоих случаях. Не знаю, как его можно улучшить :)

### Advanced

Для каждого языка посчитаем триграмы символов и обучим LaplaceProbabilityEstimator. Будем считать перплексию на каждом документе для каждого языка. Язык с минимальной перплексией и будет результатом нашего предсказания.

#### plain

In [35]:
lang_model_adv = dict()
with ZipFile('language_detection.zip', 'r') as myzip:
    for l in tqdm.tqdm(langs):
        raw = myzip.read('plain/train/{}.txt'.format(l))
        tokens = word_tokenize(raw.decode('utf-8'))
        symbols = [list(tok) for tok in tokens]
        lang_model_adv[l] = LaplaceProbabilityEstimator(NGramStorage(symbols, 2), 0.001)

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [04:42<00:00,  6.19s/it]


In [36]:
pred = dict()
with ZipFile('language_detection.zip', 'r') as myzip:
    for i in tqdm.tqdm(xrange(1, 240+1)):
        raw = myzip.read('plain/test/{}.txt'.format(i))
        tokens = word_tokenize(raw.decode('utf-8'))
        symbols = [list(tok) for tok in tokens]
        min_perp = sys.float_info.max
        forecast = None
        for lang in langs:
            perp = perplexity(lang_model_adv[lang], symbols)
            if min_perp > perp:
                min_perp = perp
                forecast = lang
        pred[i] = forecast

100%|████████████████████████████████████████████████████████████████████████████████| 240/240 [03:39<00:00,  1.31it/s]


In [37]:
errors = 0
for key in ans.keys():
    errors += 1 if ans[key]!=pred[key] else 0
print('Absolute number of errors {} (out of {} possible)'.format(errors, len(ans.keys())))
print('Relative number of errors {}%'.format(1.*errors/len(ans.keys())))

Absolute number of errors 0 (out of 240 possible)
Relative number of errors 0.0%


#### full

Все аналогично.

In [38]:
lang_model_adv = dict()
with ZipFile('language_detection.zip', 'r') as myzip:
    for l in tqdm.tqdm(langs):
        raw = myzip.read('full/train/{}.txt'.format(l))
        tokens = word_tokenize(raw.decode('utf-8'))
        symbols = [list(tok) for tok in tokens]
        lang_model_adv[l] = LaplaceProbabilityEstimator(NGramStorage(symbols, 2), 0.001)

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [04:50<00:00,  6.34s/it]


In [39]:
pred = dict()
with ZipFile('language_detection.zip', 'r') as myzip:
    for i in tqdm.tqdm(xrange(1, 240+1)):
        raw = myzip.read('full/test/{}.txt'.format(i))
        tokens = word_tokenize(raw.decode('utf-8'))
        symbols = [list(tok) for tok in tokens]
        min_perp = sys.float_info.max
        forecast = None
        for lang in langs:
            perp = perplexity(lang_model_adv[lang], symbols)
            if min_perp > perp:
                min_perp = perp
                forecast = lang
        pred[i] = forecast

100%|████████████████████████████████████████████████████████████████████████████████| 240/240 [03:45<00:00,  1.12s/it]


In [40]:
errors = 0
for key in ans.keys():
    errors += 1 if ans[key]!=pred[key] else 0
print('Absolute number of errors {} (out of {} possible)'.format(errors, len(ans.keys())))
print('Relative number of errors {}%'.format(1.*errors/len(ans.keys())))

Absolute number of errors 1 (out of 240 possible)
Relative number of errors 0.00416666666667%


Целая одна ошибка на оба варианта - было бы достойно, если бы не предыдущий результат. К слову предыдущий результат и обучается быстрее, и предсказывает. Как итог: для задачи распознавания языка при репрезентативных корпусах достаточно использовать наивные модели. Продвинутые дают сравнимое качество при больших затратах.